# Caricamento dati in mongoDB

In [46]:
# crea la lista di tratte
import os 
import json 
import pandas as pd 
path_live = '/Users/alessandro/Desktop/live/' 
lista_tratte = list()
#df = pd.DataFrame(columns= ['Giorno' 'Ritardo Totale' 'Ritardo Regionali''Ritardo NoREG''Treni in ritardo' 'Treni in anticipo' 'Treni puntuali']) 
for directory in os.listdir(path_live):  
    if(directory != ".DS_Store"): 
        for file in os.listdir(path_live + "/" + directory + "/"): 
            if file[0] != ".":
                stringa = ""
                _json = json.loads(open(path_live + "/" + directory + "/" + file, encoding="utf8").read()) 
                treno = _json[0] 
                fermate = treno["fermate"] 
                if fermate != None: 
                    for i in fermate: 
                        stringa = stringa + str(i["id"])
                    if stringa not in lista_tratte: 
                        lista_tratte.append(stringa)
lista_tratte.remove("")

In [47]:
# crea un dizionario tratta - nome tratta
import os
import json
import pandas as pd
from datetime import datetime

path_live = '/Users/alessandro/Desktop/live/'  
staz = pd.read_csv("stazioni_final.csv", sep=";")

f = dict()
count = 0
for tratta in lista_tratte:
    count += 1
    partenza = tratta[:6] 
    arrivo = tratta[len(tratta)-6:]
    p = staz[staz["_id"] == partenza]["nome_stazione"].to_string(index=False)
    a = staz[staz["_id"] == arrivo]["nome_stazione"].to_string(index=False)
    f[tratta] = p + "-" + a + "_" + str(count)

In [48]:
# crea un dizionario codice treno - nome tratta
import os
import json
import pandas as pd
path_live = '/Users/alessandro/Desktop/live/'  

l = dict()

for dir in os.listdir(path_live):   
    if(dir != ".DS_Store" ):   
        for file in os.listdir(path_live +  "/"  + dir + "/"):
            if file[0] != ".":
                stringa = ""   
                _json = json.loads(open(path_live + "/" + dir + "/" + file, encoding="utf8").read())
                treno = _json[0]
                numero = str(treno["numeroTreno"])
                categoria = treno["categoria"]
                ritardo_finale = treno["ritardo"]
                fermate = treno["fermate"]
                if categoria == None:
                    print(dir,file)
                if fermate != None:
                    codice_treno = categoria+numero
                    for i in fermate:   
                        stringa = stringa + str(i["id"])
                    if stringa != "":
                        l[codice_treno] = f[stringa]

In [ ]:
%%time
import os
import json
import pandas as pd
from datetime import datetime
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.progettoDM # Get the "twitter" database
trenitalia = db.trenitalia # Get the "tweets" collection from the "twitter" database

path_live = '/Users/alessandro/Desktop/live/'  
staz = pd.read_csv("stazioni_final.csv", sep=";")

stazioni_mancanti = dict()

for directory in os.listdir(path_live):
    #print(directory)
    my_list = list()
    if(directory[0] != ".": 
        for file in os.listdir(path_live +  "/"  + directory + "/"):
            if file[0] != ".":
                _json = json.loads(open(path_live + "/" + directory + "/" + file).read())  
                treno = _json[0]
                numero = str(treno["numeroTreno"])
                categoria = treno["categoria"]
                ritardo_finale = treno["ritardo"]
                fermate = treno["fermate"]
                origine = treno["origine"]
                destinazione = treno["destinazione"]
                codice_treno = categoria+numero
                tratta = l[codice_treno]
                if treno["tipoTreno"] == "ST":
                    my_list.append({ "codice_treno": codice_treno, 
                                "categoria": categoria,
                                "tratta": tratta,
                                "origine":origine,
                                "destinazione":destinazione,
                                "data": datetime.strptime(directory,"%d-%m-%Y"),
                                "stato": "cancellato"
                    })
                else:
                    if fermate != None:
                        if len(fermate) != 0:
                            fer = []
                            for s in fermate:
                                dati_staz = staz[staz["_id"] == s["id"]][["nome_stazione","regione","provincia","lat","lon"]]

                                if s["tipoFermata"] == "P" or s["tipoFermata"] == "F":
                                    _ritardo = s["ritardoPartenza"]

                                    if s["partenzaReale"] == None or _ritardo < -5:
                                        _ritardo = None
                                        
                                    fer.append({"id_stazione": s["id"], 
                                                "orario": datetime.fromtimestamp(s["partenza_teorica"]/1000), 
                                                "ritardo": _ritardo,
                                                "tipo_fermata": s["tipoFermata"],
                                                "nome_stazione": s["stazione"],
                                                "provincia": dati_staz["provincia"].to_string(index=False),
                                                "regione": dati_staz["regione"].to_string(index=False),
                                                "lat": dati_staz["lat"].to_string(index=False),
                                                "lon": dati_staz["lon"].to_string(index=False)
                                    })
                                elif s["tipoFermata"] == "A":
                                    
                                    _ritardo = s["ritardoArrivo"]
                                    if s["arrivoReale"] == None:
                                        _ritardo = ritardo_finale
                                        
                                    fer.append({"id_stazione": s["id"], 
                                                "orario": datetime.fromtimestamp(s["arrivo_teorico"]/1000),
                                                "ritardo": _ritardo,
                                                "tipo_fermata": s["tipoFermata"],
                                                "nome_stazione": s["stazione"],
                                                "provincia": dati_staz["provincia"].to_string(index=False),
                                                "regione": dati_staz["regione"].to_string(index=False),
                                                "lat": dati_staz["lat"].to_string(index=False),
                                                "lon": dati_staz["lon"].to_string(index=False)
                                               })
                            new_fermate = fer                     
                    else:
                        new_fermate = "non rilevate"
                    
                    if treno["tipoTreno"] != "PG":
                        my_list.append({ "codice_treno": codice_treno, 
                                    "categoria": categoria,
                                    "ritardo_finale": ritardo_finale,
                                    "data": datetime.strptime(directory,"%d-%m-%Y"),
                                    "stato": "parzialmente soppresso",
                                    "origine_teorica": treno["origineZero"],
                                    "origine": origine,
                                    "destinazione_teorica": treno["destinazioneZero"],
                                    "destinazione": destinazione,
                                    "tratta": tratta,
                                    "fermate": new_fermate
                        })
                    else:
                        my_list.append({ "codice_treno": codice_treno, 
                                    "categoria": categoria,
                                    "ritardo_finale": ritardo_finale,
                                    "data": datetime.strptime(directory,"%d-%m-%Y"),
                                    "origine":origine,
                                    "destinazione": destinazione,
                                    "tratta": tratta,
                                    "fermate": new_fermate
                        })
        
        x = trenitalia.insert_many(my_list)
        print(x)